In [2]:
import pandas as pd
import re
import sqlite3
import os

In [4]:
dump_path='xxxx'
dump_database_name='title-body.db'
db = sqlite3.connect(os.path.join(dump_path, dump_database_name))
c = db.cursor()
rq = pd.read_excel(f"/Users/jinbihui/Desktop/MS/876/lda/post-integrated.xlsx")
posts = pd.read_sql(f'select Id, Theme, Tags, QuestionDate,'+
' Coding, Energy, Device, Monitoring, CS, DT from Temp', con=db)
c.close()

In [5]:
#RQ3 tags
tags = {}
numbertags = 0
for i in range(len(posts)):
    p = posts.loc[i,'Tags']
    tag = re.findall(r'(?<=\<)(.*?)(?=\>)',p)
    numbertags += len(tag)
    temp = {}
    for t in tag:
        for ele in t.split('-'):
            if ele in temp:
                temp[ele] += 1
            else:
                temp[ele] = 1
    if len([v for v in temp.values() if int(v) > 1]):
        n = [word for word, occurrences in temp.items() if occurrences >1]
        tag = [root for root in tag if not any(node in root for node in n)]
        tag = tag+n

    for t in tag:
        if t in tags:
            tags[t] += 1
        else:
            tags[t] = 1


temp = {}
for t in tags:
    for ele in t.split('-'):
        if ele != 'c' and ele != 'on' and ele != 'es':
            if ele in temp:
                temp[ele] += 1
            else:
                temp[ele] = 1


tmp_occ={}
for d in [word for word, occurrences in temp.items() if occurrences >1]:
    try:
        int(d)
    except:
        occ = 0
        for tt in tags.keys():
            if (d in tt or tt in d) and len(tt)>=len(d):
                occ += tags[tt]
        tmp_occ[d]=occ

for d in tmp_occ:
    for tt in list(tags.keys()):
        if (d in tt or tt in d) and len(tt)>=len(d):
            del tags[tt]

    tags[d] = tmp_occ[d]
    
print("Top 10:",sorted(tags.items(), key=lambda x: x[1], reverse=True)[:10])

s = pd.DataFrame(tags.items(), columns=['Tag', 'Times']).sort_values(by='Times',ascending=False,ignore_index=True).reset_index(drop=True)
s.index = s['Tag'].values
s.drop(columns=['Tag'],inplace=True)
print("\nAll tags and their occurrences:\n",s)
print("Occurrence:", s['Times'].sum()*.8) #Pareto principle 80/20

Top 10: [('android', 274), ('ios', 114), ('energy', 97), ('battery', 75), ('performance', 67), ('java', 67), ('bluetooth', 64), ('location', 58), ('power', 55), ('linux', 55)]

All tags and their occurrences:
                  Times
android            274
ios                114
energy              97
battery             75
performance         67
...                ...
onedrive             1
office365            1
beagleboneblack      1
weka                 1
r                    1

[628 rows x 1 columns]
Occurrence: 2520.0


In [6]:
# tmpj = pd.read_excel(f'/Users/jinbihui/Desktop/MS/876/lda/posts.xlsx')

##OS
# taglabel = ['android','ios','linux','windows','macos', 'ubuntu','wear-os','ios7','ios8']
# #taglabel = ['android','ios','linux','windows','macos', 'ubuntu','ios7','ios8']
##Programming Language
# taglabel = [ 'java','c','python','c++','swift','c#','objective-c','sql','php','.net',
# 'verilog','node.js','assembly','shell','vhdl','swift3','matlab','bash','kotlin','json','jquery','css']
# #taglabel = [ 'java','c','python','c++','swift','c#','objective-c','php','assembly','vhdl','swift3']
taglabel = [ 'c','c++','c#','objective-c', 'c#']

##Device
##Device-mobile
# taglabel = ['apple','ipad','mobile','phone','tablet']
###Device-accessory
# taglabel = ['accelerometer','beacon','bluetooth','gps','ibeacon','usb','wifi','sensors']
###Device-server
# taglabel = ['server','computer']
##Device-central
# taglabel = ['microchip','microcontroller','amd','arm','core','cpu','intel','x86','msr']
##Device-graphics
# taglabel = ['gpgpu','gpu','nvidia']
##Device-storage
# taglabel = ['memory']
##Device-embedded
# taglabel = ['embedded','stm32','esp8266','esp32','arduino','pi','raspberry','fpga','xilinx','iot']

##Device-processunit
# taglabel = ['microchip','microcontroller','amd','arm','core','cpu','intel','x86','msr','gpgpu','gpu','nvidia','memory']
##Device -Central processing unit
# taglabel = ['memory','gpgpu','gpu','nvidia','microchip','microcontroller','amd','arm','core','cpu','intel','x86','msr']
##Device-all
# taglabel = [ 'bluetooth','cpu','pi','phone','gps','arduino','embedded','core','gpu','mobile','arm','wifi','memory',
# 'raspberry','intel','sensors','esp8266','usb','computer','fpga','accelerometer','stm32','ibeacon','nvidia',
# 'raspberry','fpga','xilinx''microcontroller','x86','server','apple','xilinx','amd','esp32','microchip','msr','tablet','ipad','gpgpu','beacon']

# num = 0
# flg = False
# for eachtag in tmpj['Tags'].values:
#     tag = re.findall(r'(?<=\<)(.*?)(?=\>)',eachtag)
#     for t in tag:
#         for tb in taglabel:
#             if tb in t:
#                 num+=1
#                 flg = True
#                 break
#         if flg:
#             flg=False
#             break
# print(num, round(num*100/len(tmpj),2))
tag = {}
# for theme in label:
tmp = posts
for index, e in tmp.iterrows():
    tags = {}
    for i in range(len(tmp)):
        tg = re.findall(r'(?<=\<)(.*?)(?=\>)',list(tmp['Tags'])[i])

        temp = {}
        for t in tg:
            for ele in t.split('-'):
                if ele in temp:
                    temp[ele] += 1
                else:
                    temp[ele] = 1
        if len([v for v in temp.values() if int(v) > 1]):
            n = [word for word, occurrences in temp.items() if occurrences >1]
            new_tag = [root for root in tg if not any(node in root for node in n)]
            tg = new_tag+n

        for t in range(len(tg)):
            if tg[t] in tags:
                tags[f'{tg[t]}'] +=1
            else:
                tags[f'{tg[t]}'] =1

        tags['0'] = len(tmp)
    temp = {}
    for t in tags:
        for ele in t.split('-'):
            if ele in temp:
                temp[ele] += 1
            else:
                temp[ele] = 1

    for d in [word for word, occurrences in temp.items() if occurrences >1]:
        try:
            int(d)
        except:
            if d != 'c' and d != 'on'and d != 'es':
                occ = sum( v for k, v in tags.items() if d in k )
                if occ == 0:
                    occ = sum( v for k, v in tags.items() if k in d )
                tags = {k:v for k,v in tags.items() if not d in k}
                tags[d] = occ
    tag['all'] = sorted(tags.items(), key=lambda x: x[1], reverse=True)
    
from collections import OrderedDict
fin = 0
k = 'all'
print('\n'+k)
cnt =0
tmp = {}
for v in tag[k]:


    if (v[0] == 'swift' or v[0] == 'swift3') and 'swift3' in taglabel:
        cnt += v[1]
        tmp['swift'] = tmp['swift'] + v[1] if 'swift' in tmp else v[1]
    elif (v[0] == 'ios' or v[0] == 'ios7' or v[0]=='ios8') and 'ios' in taglabel:#['ios','ios7','ios8']:
        cnt += v[1]
        tmp['ios'] = tmp['ios'] + v[1] if 'ios' in tmp else v[1]

    elif len([tl for tl in taglabel if tl == v[0]])>0:#'android' in v[0]:
        cnt += v[1]
        tmp[v[0]]=f'{round((v[1]-0)*100/len(posts),2)}%'
        # print(v[0], v[1], round((v[1]-0)*100/len(posts),2))

if 'swift' in taglabel:
    tmp['swift']= f"{round(tmp['swift']*100/len(posts),2)}%"
    new_dict = OrderedDict((k, tmp.get(k)) for k in taglabel[:len(taglabel)-1])
elif 'ios' in taglabel:
    tmp['ios']= f"{round(tmp['ios']*100/len(posts),2)}%"
    new_dict = OrderedDict((k, tmp.get(k)) for k in taglabel[:len(taglabel)-2])
else:
    new_dict = OrderedDict((k, tmp.get(k)) for k in taglabel)

fin+=cnt
print(list(new_dict.keys()))  

print(*['-' if v==None else v for k,v in list(new_dict.items())], sep = ", ")

print(fin, round(fin*100/len(posts),2))


all
['c', 'c++', 'c#', 'objective-c']
5.08%, 3.65%, 2.13%, 1.73%
124 12.59
